<a href="https://colab.research.google.com/github/Kutiolajide07/Kutiolajide07/blob/main/Sentiment_AnalysisColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Approaches to Sentiment Analysis

1. The rule-based design (popular example is the VADER)
2. A machine learning model trained on data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Twitter Sentiment Analysis

[Errno 2] No such file or directory: '/content/drive/MyDrive/Twitter Sentiment Analysis'
/content


In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
import re
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

In [5]:
! pip install twython

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [7]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer

In [8]:
!pip install polyglot

     |████████████████████████████████| 126 kB 5.4 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52577 sha256=04764d3ed32197d7fe2ffe7dc4e7154b1f2d90c744a92c57a83f3bef70b73f33
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot


In [9]:
!pip install PyICU

     |████████████████████████████████| 304 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for PyICU: filename=PyICU-2.8.1-cp37-cp37m-linux_x86_64.whl size=1375641 sha256=1bfc2a65daf489a1f5b0b95a03983508c702e65531b2508bde8ae29e5482b8bd
  Stored in directory: /root/.cache/pip/wheels/83/60/dc/4f5dffc9027a5912759aabaf918819edacfe3caffd5d31912b
Successfully built PyICU


In [10]:
!pip install pycld2

     |████████████████████████████████| 41.4 MB 1.3 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834231 sha256=3268fd84150fd9208b16afb715c03d2bc88f4f9d5d4eafaf1421fedf204c7084
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614
Successfully built pycld2


In [11]:
!pip install morfessor

In [12]:
import polyglot
from polyglot.text import Text, Word

In [13]:
df1 = pd.read_csv('/content/drive/MyDrive/sentiment analysis/uniquedata1.csv', low_memory=False)
df2 = pd.read_csv('/content/drive/MyDrive/sentiment analysis/uniquedata2.csv', low_memory=False)
df3 = pd.read_csv('/content/drive/MyDrive/sentiment analysis/unique3.csv', lineterminator='\n', low_memory=False)

In [14]:
df = pd.concat([df1, df2, df3], ignore_index=True)

In [15]:
df1.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [16]:
df.hashtags

0         ['russianukrainianwar', 'rusiaucraniaguerra', ...
1         ['russianukrainianwar', 'rusiaucraniaguerra', ...
2         ['chernobyl', 'ukrainerussiawar', 'russianukra...
3         ['ukrainerussiawar', 'standwithukraine️', 'rus...
4         ['sanctionabramovych', 'standwithukraine', 'ru...
                                ...                        
725253                                                   []
725254                                                   []
725255    ['війна', 'марш', 'наживо', 'новини', 'останні...
725256                                                   []
725257                                                   []
Name: hashtags, Length: 725258, dtype: object

In [17]:
## put hashtag in front of the hashtags

df.hashtags = df.hashtags.map(lambda x: eval(x))\
.map(lambda x: ['#' + tag for tag in x])

In [18]:
df.hashtags

0         [#russianukrainianwar, #rusiaucraniaguerra, #m...
1         [#russianukrainianwar, #rusiaucraniaguerra, #m...
2         [#chernobyl, #ukrainerussiawar, #russianukrain...
3         [#ukrainerussiawar, #standwithukraine️, #russi...
4         [#sanctionabramovych, #standwithukraine, #russ...
                                ...                        
725253                                                   []
725254                                                   []
725255    [#війна, #марш, #наживо, #новини, #останнінови...
725256                                                   []
725257                                                   []
Name: hashtags, Length: 725258, dtype: object

In [19]:
## Randomly select equal no of tweets from English and Russia

english = df[df.language == 'en'].sample(n = 22907, random_state=10)
russian = df[df.language == 'ru'].sample(n = 22907, random_state=10)
ukrain = df[df.language == 'uk']

In [20]:
tweets_eng = english[['tweet', 'hashtags']].reset_index(drop = True)
tweets_ru = russian[['tweet', 'hashtags']].reset_index(drop = True)
tweets_uk = ukrain[['tweet', 'hashtags']].reset_index(drop = True)

In [21]:
print(tweets_eng.head().iloc[3][0]); print(tweets_eng.head().iloc[3][1])

@TruthRuththe @MEGAMANTROTSKY @DukeJeopardy @RealFacelessMan @aaronjmate @JobyWarrick BLM has to do with black civil rights. Nothing to do with immigration. And if you are a supporter of BLM then how can you defend Putin's talking points while supporting a group he calls terrorists?
[]


In [22]:
before_cleaning = tweets_eng.copy()

## Data Cleaning

In [95]:
## English Tweets

tweets_eng.tweet = tweets_eng.tweet.map(lambda x: re.sub(r"http\S+", '', x))\
.map(lambda x: re.sub(r"@\S+", '', x))\
.map(lambda x: x.lower())

tweets_eng['tweet'] = tweets_eng.apply(lambda x: [word for word in x['tweet'].split() if word not in x['hashtags']], axis =1)\
.map(lambda x: ' '.join(x))

tweets_eng = tweets_eng.drop(columns = ['hashtags'])


## Russian Tweets

tweets_ru.tweet = tweets_ru.tweet.map(lambda x: re.sub(r"http\S+", '', x))\
.map(lambda x: re.sub(r"@\S+", '', x))\
.map(lambda x: x.lower())

tweets_ru['tweet'] = tweets_ru.apply(lambda x: [word for word in x['tweet'].split() if word not in x['hashtags']], axis =1)\
.map(lambda x: ' '.join(x))

tweets_ru = tweets_ru.drop(columns = ['hashtags'])

## Ukrainian Tweets

tweets_uk.tweet = tweets_uk.tweet.map(lambda x: re.sub(r"http\S+", '', x))\
.map(lambda x: re.sub(r"@\S+", '', x))\
.map(lambda x: x.lower())

tweets_uk['tweet'] = tweets_uk.apply(lambda x: [word for word in x['tweet'].split() if word not in x['hashtags']], axis =1)\
.map(lambda x: ' '.join(x))

tweets_uk = tweets_uk.drop(columns = ['hashtags'])

KeyError: ignored

In [75]:
## English

print('Before cleaning:')
print('=============')

print(before_cleaning['tweet'][1])
print(before_cleaning['tweet'][10])
print(before_cleaning['tweet'][200])
print(before_cleaning['tweet'][58])
print(before_cleaning['tweet'][96])

Before cleaning:
McDonald’s said Tuesday it is temporarily closing all of its 850 restaurants in Russia in response to the country's invasion of Ukraine. #RussianUkrainianWar #Russia #UkraineRussiaWar
@glhaize @Beaufighter_MkI @joncoopertweets 1) Context is clear: That “somebody” indiscriminately murdered innocent men, women, children/unborn babies. (2) “Gleeful” bc his death means some Ukrainians will live - at least, for now. (3) I have zero “consideration” for anyone in Putin’s army of murderers.
@McFaul 🇷🇺 - declares that 🇺🇦 is not a country 🇷🇺 - invades 🇺🇦 with ++ troops headed to Kyiv 🇷🇺 - sends multiple squads to assassinate 🇺🇦 President  @ZelenskyyUa   🇺🇸 - says that "#Putin cannot remain in power"  🇷🇺 - how dare you!  https://t.co/vEyV07zH7m
@marcbennetts1 Putin is the biggest sinners in history of russia! and Putin is the biggest mistake of the russian people!
ITEM 10: A portrait of Volodymyr Zelenskyy made out of sunflower seeds and/or sunflowers.  #IStandWithUkraine #GISH #

In [76]:
## English

print('After cleaning:')
print('=============')

print(tweets_eng['tweet'][1])
print(tweets_eng['tweet'][10])
print(tweets_eng['tweet'][200])
print(tweets_eng['tweet'][58])
print(tweets_eng['tweet'][96])

After cleaning:
mcdonald’s said tuesday it is temporarily closing all of its 850 restaurants in russia in response to the country's invasion of ukraine.
1) context is clear: that “somebody” indiscriminately murdered innocent men, women, children/unborn babies. (2) “gleeful” bc his death means some ukrainians will live - at least, for now. (3) i have zero “consideration” for anyone in putin’s army of murderers.
🇷🇺 - declares that 🇺🇦 is not a country 🇷🇺 - invades 🇺🇦 with ++ troops headed to kyiv 🇷🇺 - sends multiple squads to assassinate 🇺🇦 president 🇺🇸 - says that "#putin cannot remain in power" 🇷🇺 - how dare you!
putin is the biggest sinners in history of russia! and putin is the biggest mistake of the russian people!
item 10: a portrait of volodymyr zelenskyy made out of sunflower seeds and/or sunflowers.


## Write Sentiment Functions

Polarity: it tells us how positive/negative a tweet is. It ranges between -1 and +1. The closer to 1, the more positive the tweet and the closer to -1 the more negative the tweet. #Subjectivity: defines how opinionated a tweet is. It takes values in the range 0 to 1. The closer to 1 the more subjective and the closer to 0 the more objective.

In [26]:
# sentiment analysis using polarity from textblob
polarity = lambda tweet: TextBlob(tweet).sentiment.polarity
subjectivity = lambda tweet: TextBlob(tweet).sentiment.subjectivity  

In [27]:
tweets_eng.loc[:, 'polarity'] = tweets_eng['tweet'].apply(polarity)
tweets_eng.loc[:, 'subjectivity'] = tweets_eng['tweet'].apply(subjectivity)

In [28]:
tweets_eng.head()

,tweet,polarity,subjectivity
0,ukraine war: civilians abducted as russia trie...,-0.100000,0.400000
1,mcdonald’s said tuesday it is temporarily clos...,0.000000,0.000000
2,no one has b to stand up to putin,0.000000,0.000000
3,blm has to do with black civil rights. nothing...,0.041667,0.341667
4,down with putin! stay home stand with,-0.194444,0.288889


## Sentiment Classification

In [66]:
tweets_eng['classification'] = np.zeros(tweets_eng.shape[0])
tweets_eng['classification'] = np.where(tweets_eng['polarity'] >=0.05, 'positive', tweets_eng['classification'])
tweets_eng['classification'] = np.where(tweets_eng['polarity'] <=-0.05, 'negative', tweets_eng['classification'])
tweets_eng['classification'] = np.where((tweets_eng['polarity'] < 0.05) & (tweets_eng['polarity'] > -0.05), 'neutral', tweets_eng['classification'])

In [67]:
tweets_eng.head()

,tweet,polarity,subjectivity,classification,compound,classification2,classification3
0,ukraine war: civilians abducted as russia trie...,-0.100000,0.400000,negative,-0.8020,negative,Objective
1,mcdonald’s said tuesday it is temporarily clos...,0.000000,0.000000,neutral,0.0000,neutral,Objective
2,no one has b to stand up to putin,0.000000,0.000000,neutral,-0.2960,negative,Objective
3,blm has to do with black civil rights. nothing...,0.041667,0.341667,neutral,-0.0258,neutral,Objective
4,down with putin! stay home stand with,-0.194444,0.288889,negative,0.0000,neutral,Objective


In [68]:
tweets_eng.classification.value_counts()

neutral     9501
positive    8173
negative    5233
Name: classification, dtype: int64

based on the tweet classification above, there is a need to look deep examine the distribution to see if it is well justified.


## Examine the Neutral tweets

In [32]:
neutral_df = tweets_eng[tweets_eng.classification == 'neutral'].reset_index(drop=True)
neutral_df.head()

,tweet,polarity,subjectivity,classification
0,mcdonald’s said tuesday it is temporarily clos...,0.000000,0.000000,neutral
1,no one has b to stand up to putin,0.000000,0.000000,neutral
2,blm has to do with black civil rights. nothing...,0.041667,0.341667,neutral
3,killing putin. simple 🤷‍♂️⚒⚒⚒⚒,0.000000,0.357143,neutral
4,"feb last year ""with one stroke of a pen, zelen...",0.040179,0.269345,neutral


In [33]:
neutral_df.tweet[0]

"mcdonald’s said tuesday it is temporarily closing all of its 850 restaurants in russia in response to the country's invasion of ukraine."

In [34]:
positive_df = tweets_eng[tweets_eng.classification == 'positive'].reset_index(drop=True)
positive_df.head()

,tweet,polarity,subjectivity,classification
0,no matter how much evidence i present you with...,0.100000,0.194444,positive
1,i thought that's exactly what putin is doing w...,0.225000,0.175000,positive
2,1) context is clear: that “somebody” indiscrim...,0.109091,0.495833,positive
3,we don't give tax dollars to nazis in america ...,0.261905,0.511905,positive
4,biden did an exceptional job of uniting the wo...,0.473333,0.610000,positive


In [35]:
positive_df.tweet[10]

"why listen to 72y'old man and expecting 79y'old dumbies. patriots, you better leave the field, become friends and throw the crownmen out. at the end, mr.zelinsky may survive or not, but those ll be there alive leaving all the earth dead."

In [36]:
negative_df = tweets_eng[tweets_eng.classification == 'negative'].reset_index(drop=True)
negative_df.head()

,tweet,polarity,subjectivity,classification
0,ukraine war: civilians abducted as russia trie...,-0.100000,0.400000,negative
1,down with putin! stay home stand with,-0.194444,0.288889,negative
2,he’s an actor. he dissolved all opposition par...,-0.092727,0.590909,negative
3,you left out gaslighting. you would have to as...,-0.350000,0.491667,negative
4,"given how much is made of left-wing groups ""de...",-0.137500,0.340000,negative


In [37]:
negative_df.tweet[0]

'ukraine war: civilians abducted as russia tries to assert control'

In [38]:
## Using compound from VADER

vader = SentimentIntensityAnalyzer()
vader.polarity_scores(negative_df.tweet[0])

{'compound': -0.802, 'neg': 0.474, 'neu': 0.526, 'pos': 0.0}

In [39]:
Vader = lambda tweet: vader.polarity_scores(tweet)['compound']

In [40]:
tweets_eng.loc[:, 'compound'] = tweets_eng['tweet'].apply(Vader)

In [41]:
tweets_eng

,tweet,polarity,subjectivity,classification,compound
0,ukraine war: civilians abducted as russia trie...,-0.100000,0.400000,negative,-0.8020
1,mcdonald’s said tuesday it is temporarily clos...,0.000000,0.000000,neutral,0.0000
2,no one has b to stand up to putin,0.000000,0.000000,neutral,-0.2960
3,blm has to do with black civil rights. nothing...,0.041667,0.341667,neutral,-0.0258
4,down with putin! stay home stand with,-0.194444,0.288889,negative,0.0000
...,...,...,...,...,...
22902,biden said it straight putin must go. don’t wa...,0.100000,0.200000,positive,0.2263
22903,putin’s ‘war-on-woke’ in mariupol just now.,0.000000,0.000000,neutral,0.0000
22904,think they'll offer an apology for making a fr...,0.000000,0.000000,neutral,0.0516
22905,one month into russia's military operation in ...,0.200000,0.300000,positive,-0.5984


In [42]:
tweets_eng['classification2'] = np.zeros(tweets_eng.shape[0])
tweets_eng['classification2'] = np.where(tweets_eng['compound'] >=0.05, 'positive', tweets_eng['classification2'])
tweets_eng['classification2'] = np.where(tweets_eng['compound'] <=-0.05, 'negative', tweets_eng['classification2'])
tweets_eng['classification2'] = np.where((tweets_eng['compound'] < 0.05) & (tweets_eng['compound'] > -0.05), 'neutral', tweets_eng['classification2'])

In [43]:
tweets_eng

,tweet,polarity,subjectivity,classification,compound,classification2
0,ukraine war: civilians abducted as russia trie...,-0.100000,0.400000,negative,-0.8020,negative
1,mcdonald’s said tuesday it is temporarily clos...,0.000000,0.000000,neutral,0.0000,neutral
2,no one has b to stand up to putin,0.000000,0.000000,neutral,-0.2960,negative
3,blm has to do with black civil rights. nothing...,0.041667,0.341667,neutral,-0.0258,neutral
4,down with putin! stay home stand with,-0.194444,0.288889,negative,0.0000,neutral
...,...,...,...,...,...,...
22902,biden said it straight putin must go. don’t wa...,0.100000,0.200000,positive,0.2263,positive
22903,putin’s ‘war-on-woke’ in mariupol just now.,0.000000,0.000000,neutral,0.0000,neutral
22904,think they'll offer an apology for making a fr...,0.000000,0.000000,neutral,0.0516,positive
22905,one month into russia's military operation in ...,0.200000,0.300000,positive,-0.5984,negative


In [44]:
tweets_eng.classification2.value_counts()

negative    10917
positive     7559
neutral      4431
Name: classification2, dtype: int64

In [45]:
tweets_eng[tweets_eng.classification2 == 'negative'].reset_index()['tweet'][0]

'ukraine war: civilians abducted as russia tries to assert control'

In [46]:
tweets_eng[tweets_eng.classification2 == 'negative'].reset_index()['tweet'][500]

"'if nato had not expanded, mr. putin would be menacing not just ukraine but the baltic states and likely all of eastern europe.' hillary clinton: madeleine albright warned us, and she was right"

In [47]:
tweets_eng[tweets_eng.classification2 == 'negative'].reset_index()['tweet'][500]

"'if nato had not expanded, mr. putin would be menacing not just ukraine but the baltic states and likely all of eastern europe.' hillary clinton: madeleine albright warned us, and she was right"

In [48]:
tweets_eng[tweets_eng.classification2 == 'positive'].reset_index()['tweet'][965]

'to art break #7-bonus good night vladyslav shereshevsky- kyiv artist lh-“levitan”(2013) isaac levitan was a russian classical landscape painter (1860-1900) rh- vladyslav at art showing in front of one of his works'

In [49]:
tweets_eng[tweets_eng.classification2 == 'positive'].reset_index()['tweet'][500]

'should be painfully clear to everyone inside and outside russia 🇷🇺 that the only thing at stake is the current leadership in russia. the country and its citizens will be better off the sooner putin leaves'

In [50]:
## Inspect Subjectivity

In [51]:
tweets_eng['classification3'] = np.zeros(tweets_eng.shape[0])
tweets_eng['classification3'] = np.where(tweets_eng['subjectivity'] >0.5, 'Subjective', tweets_eng['classification3'])
tweets_eng['classification3'] = np.where(tweets_eng['subjectivity'] <=0.5, 'Objective', tweets_eng['classification3'])

In [52]:
tweets_eng['classification3'].value_counts()

Objective     15888
Subjective     7019
Name: classification3, dtype: int64

In [53]:
pd.crosstab(tweets_eng['classification3'], tweets_eng['classification2'])

classification2,negative,neutral,positive
classification3,,,
Objective,7316,3899,4673
Subjective,3601,532,2886


## Russian Analysis

In [87]:
russian

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Unnamed: 0,trans_dest\r
396280,1507828554674982921,1507778488442490892,2022-03-26 21:15:16 GMT Standard Time,2022-03-26,21:15:16,100,1262590886271234056,promete43774782,Prometey,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'givemeatalk', 'name': 'unple...",NaN,NaN,NaN,NaN,NaN,\r
608197,1506961944510672906,1506961944510672906,2022-03-24 11:51:40 GMT Standard Time,2022-03-24,11:51:40,100,1281277547251142657,gornyak75,gornyak75_80,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,\r
606702,1507007230876393472,1506653191504547840,2022-03-24 14:51:37 GMT Standard Time,2022-03-24,14:51:37,100,1505959800387121162,novorodovskij,Додык,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'ZelenskyyUa', 'name': 'Волод...",NaN,NaN,NaN,NaN,NaN,\r
397688,1507774435637567497,1506496053582905349,2022-03-26 17:40:13 GMT Standard Time,2022-03-26,17:40:13,100,219184428,dubluwn,Bill Natale,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'StockholmQ3', 'name': 'Vladi...",NaN,NaN,NaN,NaN,NaN,\r
573027,1508166546987884549,1508166542738923522,2022-03-27 20:38:19 GMT Summer Time,2022-03-27,20:38:19,100,1388100685229658112,pepelkajj,овердью 🕊️ #нетвойне,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,\r
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408034,1507263710158782469,1507239222444494848,2022-03-25 07:50:46 GMT Standard Time,2022-03-25,07:50:46,100,3053170378,i_army_org,ІнфоВійська України/Ukraine,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'pavelnshytov', 'name': '🇺🇦 П...",NaN,NaN,NaN,NaN,NaN,\r
383656,1508566464793620482,1508390223226580997,2022-03-28 23:07:27 GMT Summer Time,2022-03-28,23:07:27,100,870007425281253376,oekolf,Doctor Ratess,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'NovaShpakova', 'name': '☭ No...",NaN,NaN,NaN,NaN,NaN,\r
404256,1507401864052051971,1507401864052051971,2022-03-25 16:59:45 GMT Standard Time,2022-03-25,16:59:45,100,1408516110635388929,ricksan41606936,Rick Sanchez,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,\r
385188,1508485571924090882,1508485571924090882,2022-03-28 17:46:01 GMT Summer Time,2022-03-28,17:46:01,100,82790668,monk696,"Vytis ""Chase"" Maleckas (Małecki) Brunius ▲",NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,\r


In [88]:
russian.tweet

396280    @givemeatalk @max_katz Путин, в отличие от вас...
608197    #Ukraine Президент @ZelenskyyUa   должен награ...
606702          @ZelenskyyUa @BorisJohnson Хорошая новость.
397688    @StockholmQ3 @element__pr @KremlinRussia_E Ваш...
573027    люди умирают у моих ровесников и меня нет буду...
                                ...                        
408034    @pavelnshytov @rianru "россия - это путин, пут...
383656    @NovaShpakova @DNNEuropa @__MaXXiM__ Неважно, ...
404256    Автобус в Москве со мной разговаривает. Путин ...
385188    #Путин #Украина #войнаЧто теперь будет с Путин...
564380    @Unreal2131 @Kris_Bale @irina_vanpatten @ilyak...
Name: tweet, Length: 22907, dtype: object

In [94]:
text = Text('марш')
print('Language Detected: Code={}\n'.format(text.language.code, text.language.name))

Detector is not able to detect the language reliably.


Language Detected: Code=ru



In [54]:
!polyglot download sentiment2.ru

[polyglot_data] Downloading package sentiment2.ru to
[polyglot_data]     /root/polyglot_data...


In [55]:
def Polarity2(tweet):

  try:
    polarity = Text(tweet).polarity
  except:
    polarity = 0
  return polarity

In [56]:
tweets_ru.loc[:, 'polarity'] = tweets_ru['tweet'].apply(Polarity2)

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [57]:
tweets_ru

,tweet,polarity
0,"путин, в отличие от вас, это прекрасно понимае...",0.6
1,президент должен наградить russia today медаль...,0.0
2,хорошая новость.,0.0
3,"ваш лидер был на футбольном стадионе, битком н...",1.0
4,люди умирают у моих ровесников и меня нет буду...,0.0
...,...,...
22902,"""россия - это путин, путин - это россия"" (с)",0.0
22903,"неважно, что вырвано из контекста, главное, чт...",0.0
22904,автобус в москве со мной разговаривает. путин ...,1.0
22905,теперь будет с путиным: сценарии от западных э...,0.0


In [58]:
tweets_ru['classification'] = np.zeros(tweets_ru.shape[0])
tweets_ru['classification'] = np.where(tweets_ru['polarity'] >=0.05, 'positive', tweets_ru['classification'])
tweets_ru['classification'] = np.where(tweets_ru['polarity'] <=-0.05, 'negative', tweets_ru['classification'])
tweets_ru['classification'] = np.where((tweets_ru['polarity'] < 0.05) & (tweets_ru['polarity'] > -0.05), 'neutral', tweets_ru['classification'])

In [59]:
tweets_ru.head()

,tweet,polarity,classification
0,"путин, в отличие от вас, это прекрасно понимае...",0.6,positive
1,президент должен наградить russia today медаль...,0.0,neutral
2,хорошая новость.,0.0,neutral
3,"ваш лидер был на футбольном стадионе, битком н...",1.0,positive
4,люди умирают у моих ровесников и меня нет буду...,0.0,neutral


In [60]:
tweets_ru.classification.value_counts()

neutral     12251
positive     6465
negative     4191
Name: classification, dtype: int64

## Ukrain Analysis

In [81]:
ukrain

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Unnamed: 0,trans_dest\r
138,1502021161131118595,1502021161131118592,2022-03-10 13:38:45-07:00,2022-03-10 00:00:00,13:38:45,-700,1319608696624893952,krymchanin32,Krymchanin32,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
264,1502000889472684033,1502000889472684032,2022-03-10 12:18:12-07:00,2022-03-10 00:00:00,12:18:12,-700,1338033786995544065,patsanyan,Eduard Patsanyan,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
308,1501993514841915406,1501993514841915392,2022-03-10 11:48:54-07:00,2022-03-10 00:00:00,11:48:54,-700,1499830603528777729,saveyourselfzt,Save Yourself 🇺🇦,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
370,1501978569613975552,1501978569613975552,2022-03-10 10:49:31-07:00,2022-03-10 00:00:00,10:49:31,-700,2761813215,mkostevska,Margaryta_B,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
383,1501974896800321546,1501974896800321536,2022-03-10 10:34:55-07:00,2022-03-10 00:00:00,10:34:55,-700,575076625,vinrova,Svitlana,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725253,1502292153015357447,1502292153015357440,2022-03-11 14:35:35 GMT Standard Time,2022-03-11 00:00:00,14:35:35,0,1279459946724306944,rightsgerman,Civil rights activist and German politician,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,\r
725254,1502291086231977992,1502291086231977984,2022-03-11 14:31:20 GMT Standard Time,2022-03-11 00:00:00,14:31:20,0,1467405441114193922,vladislavka1488,владіславка🕸🇺🇦,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,\r
725255,1502290151443247115,1502290151443247104,2022-03-11 14:27:37 GMT Standard Time,2022-03-11 00:00:00,14:27:37,0,270577554,liberoreporter,LiberoReporter,NaN,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,\r
725256,1502253379732529156,1502040704188653568,2022-03-11 12:01:30 GMT Standard Time,2022-03-11 00:00:00,12:01:30,0,1497722802153177089,pawluszka89,Paweł G,NaN,...,NaN,NaN,NaN,"[{'screen_name': 'MadMags0', 'name': 'Magda M'...",NaN,NaN,NaN,NaN,NaN,\r


In [82]:
ukrain.tweet

138       Коротко по Украине. 10.03.2022. #Донбасс, #вой...
264       Всі, хто кликав росію і путіна, ви чому сидите...
308       ⚡️Месники: українська версія🇺🇦  #StandWithUkra...
370       Після війни я обов‘язково… 💛💙 Обов‘язково пове...
383       Захід повинен закрити небо над Україною💛💙🙏 #Pe...
                                ...                        
725253    Українська війна: Володимир Зеленський бачить ...
725254                Російсько-білорусько-українська війна
725255    LIVE Українська війна: ядерна небезпека, досі ...
725256    @MadMags0 @nexta_tv якщо це неправда, то вона ...
725257    @nexta_tv якщо це неправда, то вона в Туреччин...
Name: tweet, Length: 22907, dtype: object

In [86]:
text = Text('Українська війна')
print('Language Detected: Code={}\n'.format(text.language.code, text.language.name))

Language Detected: Code=uk



In [69]:
!polyglot download sentiment2.uk

[polyglot_data] Downloading package sentiment2.uk to
[polyglot_data]     /root/polyglot_data...
[polyglot_data]   Package sentiment2.uk is already up-to-date!


In [70]:
tweets_uk.loc[:, 'polarity'] = tweets_uk['tweet'].apply(Polarity2)

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to dete

In [71]:
tweets_uk['classification'] = np.zeros(tweets_uk.shape[0])
tweets_uk['classification'] = np.where(tweets_uk['polarity'] >=0.05, 'positive', tweets_uk['classification'])
tweets_uk['classification'] = np.where(tweets_uk['polarity'] <=-0.05, 'negative', tweets_uk['classification'])
tweets_uk['classification'] = np.where((tweets_uk['polarity'] < 0.05) & (tweets_uk['polarity'] > -0.05), 'neutral', tweets_uk['classification'])

In [72]:
tweets_uk.classification.value_counts()

neutral     12563
positive     5779
negative     4565
Name: classification, dtype: int64

In [73]:
## Possible alternatives libraries

# 1. indicoio
# 2. vader-multi